In [1]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
# a_pattern = 'SI06322.Chn'

In [2]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada
from sklearn import linear_model

import pandas as pd
from random import randrange
import plotly.graph_objects as go

from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [3]:
ogra = Ograyspy()
print(ogra.info_node)
to_be_found = 'Genie_Transfer'
print('\nExec ogra.define_files_folder(to_be_found):')
ogra.define_files_folder(to_be_found)
print(ogra.pkl_folder_files)
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
del ogra

Found folder name:  /home/marcelofm/snap
Partes:  ('/', 'home', 'marcelofm', 'snap')
No. spec files:  0
marcelofm-X540LA

Exec ogra.define_files_folder(to_be_found):
Found folder name:  /home/marcelofm/Nextcloud/Genie_Transfer
Partes:  ('/', 'home', 'marcelofm', 'Nextcloud', 'Genie_Transfer')
No. spec files:  6891
Linuxmarcelofm-X540LA.pkl


In [4]:
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Linux-6.2.0-23-generic-x86_64-with-glibc2.37,x86_64,Linux,marcelofm-X540LA,/home/marcelofm,/home/marcelofm/Nextcloud/Genie_Transfer,6891,0,,,[/home/marcelofm/Nextcloud/Genie_Transfer/BGNO...,"[BGNOVO.Chn, testes_lowbg_planar/Ba133 Planar....","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",Linuxmarcelofm-X540LA.pkl


In [5]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [6]:
reduc_nms

['BGNOVO.Chn',
 'testes_lowbg_planar/Ba133 Planar.Chn',
 'testes_lowbg_planar/576ALoBG.Chn',
 'testes_lowbg_planar/Eu152 Planar.Chn',
 'testes_lowbg_planar/2022-dez-09_Co-60_colimado.Chn',
 'testes_lowbg_planar/2022-dez-09_Co-60_bequer.Chn',
 'PNI_2022-04/Dy Abr-22 - digimed 1 - Det P.Chn',
 'PNI_2022-04/Dy Abr-22 - digimed 2 - Det P.Chn',
 'PNI_2022-04/Dy Abr-22 - digimed 3 - Det P.Chn',
 'Luiz/Ra226/teste 1/Branco B 20ks.Chn',
 'Luiz/Ra226/teste 1/PNI D 20ks.Chn',
 'Luiz/Ra226/teste 1/PNI E 20ks.Chn',
 'Luiz/Ra226/teste 1/Branco A 20ks.Chn',
 'Luiz/Ra226/teste 1/Geiser C 20ks.Chn',
 'Luiz/Ra226/teste 1/Pote vazio 20ks.Chn',
 'Si/SI2004/si19504.CHN',
 'Si/SI2004/si12104.CHN',
 'Si/SI2004/si19104.CHN',
 'Si/SI2004/si12204.CHN',
 'Si/SI2004/si16904.CHN',
 'Si/SI2004/CCI13101.chn',
 'Si/SI2004/si12904.CHN',
 'Si/SI2004/si034042.chn',
 'Si/SI2004/SI11804.chn',
 'Si/SI2004/SI14704.CHN',
 'Si/SI2004/SI13804.CHN',
 'Si/SI2004/SI20804.chn',
 'Si/SI2004/si20304.chn',
 'Si/SI2004/si15204.CHN',


In [7]:
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06322.Chn'

In [8]:
f_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)
f_name, reduced_f_name

Existing:
Achou! indice=903, nomearq = Si/SI2022/SI06322.Chn
Final choices:
spectra_path: /home/marcelofm/Nextcloud/Genie_Transfer
a_spec_name: /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.Chn
reduced_f_name: Si/SI2022/SI06322.Chn


(PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.Chn'),
 'Si/SI2022/SI06322.Chn')

In [9]:
nucl_iear1_df = pd.read_pickle('nucl_iear1_list.pkl')
nucl_iear1_df

energy  unc_en  intensity    unc_i start_level_energy  \
24na  0     472.2023  0.0014   99.90000    0.001           472.2071   
      1     996.8300     0.1    0.00210   0.0002            5235.21   
      2    1368.6250   0.005   99.99400    0.002           1368.667   
      3    2754.0080   0.011   99.86700     0.01           4122.844   
      4    2871.0000       1    0.00025  0.00004            4238.38   
...              ...     ...        ...      ...                ...   
228Ac 267    89.9540            2.01000     0.19                      
      268    93.3470            3.20000      0.3                      
      269   105.5660            1.16000     0.11                      
      270   106.8940            1.54000     0.15                      
      271   108.5800            0.39000     0.04                      

          end_level_energy multipolarity mixing_ratio unc_mr conversion_coeff  \
24na  0                  0          [M3]                 NaN          4.69E-4   
      1            4238.38                               NaN                    
      2                  0            E2                 NaN           1.3E-5   
      3           1368.667            E2                 NaN                    
      4           1368.667         M1+E2          -23      9                    
...                    ...           ...          ...    ...              ...   
228Ac 267                                                                       
      268                                                                       
      269                                                                       
      270                                                                       
      271                                                                       

           ... unc_d         q  unc_q d_z  d_n  d_symbol  \
24na  0    ...                    NaN  11   13        Na   
      1    ...        5515.677   21.0  12   12        Mg   
      2    ...        5515.677   21.0  12   12        Mg   
      3    ...        5515.677   21.0  12   12        Mg   
      4    ...        5515.677   21.0  12   12        Mg   
...        ...   ...       ...    ...  ..  ...       ...   
228Ac 267  ...          2123.8   26.0  90  138        Th   
      268  ...          2123.8   26.0  90  138        Th   
      269  ...          2123.8   26.0  90  138        Th   
      270  ...          2123.8   26.0  90  138        Th   
      271  ...          2123.8   26.0  90  138        Th   

          ensdf_publication_cut-off  \
24na  0                 31-Mar-2022   
      1                 31-Mar-2022   
      2                 31-Mar-2022   
      3                 31-Mar-2022   
      4                 31-Mar-2022   
...                             ...   
228Ac 267               31-Dec-2012   
      268               31-Dec-2012   
      269               31-Dec-2012   
      270               31-Dec-2012   
      271               31-Dec-2012   

                                            ensdf_authors  Extraction_date  \
24na  0    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
      1    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
      2    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
      3    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
      4    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
...                                                   ...              ...   
228Ac 267                              KHALIFEH ABUSALEEM       2023-06-15   
      268                              KHALIFEH ABUSALEEM       2023-06-15   
      269                              KHALIFEH ABUSALEEM       2023-06-15   
      270                              KHALIFEH ABUSALEEM       2023-06-15   
      271                              KHALIFEH ABUSALEEM       2023-06-15   

          nuclide_name  
24na  0           24na  
      1    

In [10]:
# 2023-Jun-15: Setting gamma lines/ranges to dismiss in the analysis
df1 = nucl_iear1_df
df1["is_to_consider"] = True
df1.loc[((df1.energy > 509) & (df1.energy < 513)) | (df1.energy < 100), "is_to_consider"] = False
nucl_iear1_df

energy  unc_en  intensity    unc_i start_level_energy  \
24na  0     472.2023  0.0014   99.90000    0.001           472.2071   
      1     996.8300     0.1    0.00210   0.0002            5235.21   
      2    1368.6250   0.005   99.99400    0.002           1368.667   
      3    2754.0080   0.011   99.86700     0.01           4122.844   
      4    2871.0000       1    0.00025  0.00004            4238.38   
...              ...     ...        ...      ...                ...   
228Ac 267    89.9540            2.01000     0.19                      
      268    93.3470            3.20000      0.3                      
      269   105.5660            1.16000     0.11                      
      270   106.8940            1.54000     0.15                      
      271   108.5800            0.39000     0.04                      

          end_level_energy multipolarity mixing_ratio unc_mr conversion_coeff  \
24na  0                  0          [M3]                 NaN          4.69E-4   
      1            4238.38                               NaN                    
      2                  0            E2                 NaN           1.3E-5   
      3           1368.667            E2                 NaN                    
      4           1368.667         M1+E2          -23      9                    
...                    ...           ...          ...    ...              ...   
228Ac 267                                                                       
      268                                                                       
      269                                                                       
      270                                                                       
      271                                                                       

           ...         q  unc_q  d_z  d_n d_symbol  ensdf_publication_cut-off  \
24na  0    ...              NaN   11   13       Na                31-Mar-2022   
      1    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      2    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      3    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      4    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
...        ...       ...    ...  ...  ...      ...                        ...   
228Ac 267  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      268  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      269  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      270  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      271  ...    2123.8   26.0   90  138       Th                31-Dec-2012   

                                            ensdf_authors Extraction_date  \
24na  0    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      1    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      2    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      3    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      4    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
...                                                   ...             ...   
228Ac 267                              KHALIFEH ABUSALEEM      2023-06-15   
      268                              KHALIFEH ABUSALEEM      2023-06-15   
      269                              KHALIFEH ABUSALEEM      2023-06-15   
      270                              KHALIFEH ABUSALEEM      2023-06-15   
      271                              KHALIFEH ABUSALEEM      2023-06-15   

           nuclide_name is_to_consider  
24na  0            24na           True  
      1            24na           True  
      2            24na           True  
      3            24na           True  
      4            24na           True  
...                 ...            ...  
228

In [11]:
a_spec = Spec(f_name, reduced_f_name)

In [12]:
dir(a_spec)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'channel_energy_calib',
 'curr_h_win',
 'det_descr',
 'f_name',
 'final_composed_baseline',
 'generate_pandas_dataframe',
 'lv_time',
 'net_spec_ser_an',
 'origin_spec_ser_an',
 'pkl_file',
 'reduced_f_name',
 'rl_time',
 'sam_descr',
 'source_datetime',
 'spec_io',
 'start_datetime',
 'sufx',
 'total_analysis']

In [13]:
orig_ser = a_spec.origin_spec_ser_an
vars(orig_ser)

{'mix_regions': None,
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0, 0, 0, ..., 3, 1, 2]),
 'nzero': array([False, False, False, ...,  True,  True,  True]),
 'n_ch': 4096,
 'final_baseline': None,
 'x_s': array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
        4.095e+03]),
 'chans_nzero': array([  26.,   27.,   28., ..., 4093., 4094., 4095.]),
 'counts_nzero': array([ 11, 221, 380, ...,   3,   1,   2]),
 'unc_y': array([ 3.31662479, 14.86606875, 19.49358869, ...,  1.73205081,
         1.        ,  1.41421356]),
 'given_variance': array([0, 0, 0, ..., 3, 1, 2]),
 'y_smoothed': array([0, 0, 0, ..., 3, 1, 2]),
 'fft_s': None,
 'is_reg': array([False, False, False, ..., False, False, False]),
 'xs_all_mplets': [],
 'ys_all_mplets': [],
 'ys_all_steps': [],
 'chans_in_multiplets_list': [],
 'calculated_step_counts': [],
 'xs_bl_in_reg': array([], dtype=float64),
 'ys_bl_in_reg': array([], dtype=float64),
 'xs_bl_out_reg': array([], dtype=float64),
 'ys_b

In [14]:
a_spec.total_analysis(gener_dataframe=True)
# A opção acima, gener_dataframe=True,  grava o pkl_file do espectro
vars(a_spec)

Starting Spec.total_analysis...
Finish Spec.total_analysis!


{'net_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7f66fb57c610>,
 'final_composed_baseline': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7f66fb57c0d0>,
 'f_name': PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.Chn'),
 'reduced_f_name': 'Si/SI2022/SI06322.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.58,
 'source_datetime': None,
 'pkl_file': PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.pkl'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7f66fcd473d0>,
 'start_datetime': datetime.datetime(2022, 8, 4, 19, 42, 23),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-063/22 - Agua da Piscina do Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x7f66fb7f86d0>}

In [15]:
vars(orig_ser)

{'mix_regions': array([[  24,   57],
        [ 114,  180],
        [ 236,  267],
        [ 278,  285],
        [ 328,  341],
        [ 352,  363],
        [ 375,  386],
        [ 393,  400],
        [ 443,  450],
        [ 458,  465],
        [ 477,  500],
        [ 524,  535],
        [ 537,  544],
        [ 552,  567],
        [ 576,  589],
        [ 596,  607],
        [ 612,  636],
        [ 669,  674],
        [ 678,  685],
        [ 711,  720],
        [ 721,  745],
        [ 759,  768],
        [ 800,  807],
        [ 812,  827],
        [ 834,  851],
        [ 926,  951],
        [ 954,  984],
        [ 989, 1018],
        [1047, 1054],
        [1076, 1089],
        [1102, 1138],
        [1155, 1177],
        [1182, 1202],
        [1213, 1232],
        [1249, 1266],
        [1275, 1294],
        [1298, 1317],
        [1326, 1350],
        [1366, 1379],
        [1406, 1425],
        [1447, 1462],
        [1491, 1504],
        [1534, 1547],
        [1580, 1598],
        [1637, 16

In [16]:
net_ser = a_spec.net_spec_ser_an
vars(net_ser)

{'mix_regions': array([[  25,   57],
        [ 122,  134],
        [ 139,  150],
        [ 231,  240],
        [ 393,  400],
        [ 479,  498],
        [ 525,  534],
        [ 577,  588],
        [ 712,  719],
        [ 834,  851],
        [ 955,  966],
        [ 998, 1009],
        [1076, 1089],
        [1108, 1123],
        [1125, 1136],
        [1156, 1169],
        [1193, 1200],
        [1250, 1265],
        [1326, 1341],
        [1367, 1378],
        [1448, 1461],
        [1492, 1503],
        [1534, 1547],
        [1580, 1598],
        [1827, 1848],
        [1919, 1934],
        [2027, 2040],
        [2180, 2195],
        [2264, 2281],
        [2390, 2407],
        [2465, 2476],
        [2606, 2623],
        [2832, 2843],
        [2888, 2903],
        [3025, 3036],
        [3438, 3463],
        [3607, 3624],
        [4001, 4028]]),
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0.        , 0.        , 0.        , ..., 2.0245828 , 0.        ,
        1.43130251])

In [17]:
print(a_spec.pkl_file)

/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.pkl


In [18]:
a_spec.pkl_file

PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.pkl')

In [19]:
# Só para verificar se o pkl_file pode ser recuperado:
read_analysis_spec = pd.read_pickle(a_spec.pkl_file)

In [20]:
read_orig = read_analysis_spec['origin_spec_ser_an'][0]
orig_ser = read_orig
vars(orig_ser)

{'mix_regions': array([[  24,   57],
        [ 114,  180],
        [ 236,  267],
        [ 278,  285],
        [ 328,  341],
        [ 352,  363],
        [ 375,  386],
        [ 393,  400],
        [ 443,  450],
        [ 458,  465],
        [ 477,  500],
        [ 524,  535],
        [ 537,  544],
        [ 552,  567],
        [ 576,  589],
        [ 596,  607],
        [ 612,  636],
        [ 669,  674],
        [ 678,  685],
        [ 711,  720],
        [ 721,  745],
        [ 759,  768],
        [ 800,  807],
        [ 812,  827],
        [ 834,  851],
        [ 926,  951],
        [ 954,  984],
        [ 989, 1018],
        [1047, 1054],
        [1076, 1089],
        [1102, 1138],
        [1155, 1177],
        [1182, 1202],
        [1213, 1232],
        [1249, 1266],
        [1275, 1294],
        [1298, 1317],
        [1326, 1350],
        [1366, 1379],
        [1406, 1425],
        [1447, 1462],
        [1491, 1504],
        [1534, 1547],
        [1580, 1598],
        [1637, 16

In [21]:
read_net = read_analysis_spec['net_spec_ser_an'][0]
net_ser = read_net
vars(net_ser)

{'mix_regions': array([[  25,   57],
        [ 122,  134],
        [ 139,  150],
        [ 231,  240],
        [ 393,  400],
        [ 479,  498],
        [ 525,  534],
        [ 577,  588],
        [ 712,  719],
        [ 834,  851],
        [ 955,  966],
        [ 998, 1009],
        [1076, 1089],
        [1108, 1123],
        [1125, 1136],
        [1156, 1169],
        [1193, 1200],
        [1250, 1265],
        [1326, 1341],
        [1367, 1378],
        [1448, 1461],
        [1492, 1503],
        [1534, 1547],
        [1580, 1598],
        [1827, 1848],
        [1919, 1934],
        [2027, 2040],
        [2180, 2195],
        [2264, 2281],
        [2390, 2407],
        [2465, 2476],
        [2606, 2623],
        [2832, 2843],
        [2888, 2903],
        [3025, 3036],
        [3438, 3463],
        [3607, 3624],
        [4001, 4028]]),
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0.        , 0.        , 0.        , ..., 2.0245828 , 0.        ,
        1.43130251])

In [22]:
# 2023-May-26
# Ok! Espectro gravado, depois lido, então vamos prosseguir
# com a identificação dos nuclídeos.
# Agora, posso ler e analisar um espectro, gravá-lo e depois, em outro momento,
# ler o pkl com a análise.

In [23]:
x_nz = orig_ser.chans_nzero
y_nz = orig_ser.counts_nzero

In [24]:
chans = net_ser.x_s
ys_net_counts = net_ser.y_s
peaks_net = net_ser.pk_parms.peaks
peaks_orig = orig_ser.pk_parms.peaks
counts = orig_ser.y_s
chans_nzero = orig_ser.chans_nzero
counts_nzero = orig_ser.counts_nzero
unc_y = orig_ser.unc_y
eval_y = orig_ser.y_smoothed
eval_bl = orig_ser.eval_baseline
fin_bl = orig_ser.final_baseline
inis = orig_ser.pk_parms.propts['left_bases']
fins = orig_ser.pk_parms.propts['right_bases']

In [25]:
# 2023-Jun-15
# CRUCIAL step: take the dict net_ser.pk_parms' keys/values and organize them as a pd.Dataframe
vars_pkprms = vars(net_ser.pk_parms)
keys_to_get = ['peaks', 'fwhm_centr', 'rough_sums', 'centroids', 'variances']
prep_for_dict = [(key, vars_pkprms[key]) for key in keys_to_get]
pks_dict = dict(prep_for_dict)
peaks_df = pd.DataFrame.from_dict(pks_dict)
peaks_df

,peaks,fwhm_centr,rough_sums,centroids,variances
0,31,31.015664,1623.153269,31.188541,10243.062773
1,42,43.800501,1430.111502,44.293327,21540.729150
2,125,124.694475,1631.001467,125.249005,6686.998533
3,128,128.281527,2933.284105,127.659903,10096.715895
4,144,144.469761,1383.667055,144.548235,7786.332945
5,235,235.390313,332.094692,235.142846,7906.898565
6,396,396.020142,390.814605,396.263267,5348.731117
7,488,487.820179,533.344351,488.149398,10970.082038
8,529,529.359782,297.023953,529.165375,4995.087932
9,582,581.614266,484.212268,581.772500,5194.024894


In [26]:
pr_pk = dict([('pk_hei',net_ser.pk_parms.propts['peak_heights']),
              ('lf_thr',net_ser.pk_parms.propts['left_thresholds']),
              ('rg_thr',net_ser.pk_parms.propts['right_thresholds']),
              ('promns',net_ser.pk_parms.propts['prominences']),
              ('lf_bas',net_ser.pk_parms.propts['left_bases']),
              ('rg_bas',net_ser.pk_parms.propts['right_bases']),
              ('widths',net_ser.pk_parms.propts['widths']),
              ('wi_hei',net_ser.pk_parms.propts['width_heights']),
              ('lf_ips',net_ser.pk_parms.propts['left_ips']),
              ('rg_ips',net_ser.pk_parms.propts['right_ips'])])

In [27]:
teste_df = pd.DataFrame(pr_pk)
teste_df

,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips
0,552.373258,157.062347,216.573947,552.373258,28,36,2.826510,276.186629,29.602409,32.428919
1,178.084376,12.920504,42.207336,178.084376,36,56,6.829875,89.042188,40.385563,47.215438
2,449.256447,71.048679,194.556970,194.556970,119,126,1.611050,351.977962,123.888950,125.500000
3,890.961140,484.133295,239.382450,890.961140,119,134,2.403376,445.480570,127.079839,129.483215
4,520.177021,385.848439,76.998316,520.177021,140,153,2.287660,260.088510,143.325931,145.613591
5,132.552658,91.285233,0.699236,132.552658,224,238,2.232697,66.276329,234.273964,236.506662
6,216.078312,156.591931,147.953356,216.078312,394,404,1.420165,108.039156,395.310059,396.730224
7,99.343812,45.262762,5.755925,99.343812,482,492,4.616647,49.671906,485.511856,490.128502
8,113.048598,74.117583,42.991935,113.048598,526,532,2.244825,56.524299,528.237370,530.482195
9,167.577046,11.007271,92.194617,167.577046,579,585,2.589113,83.788523,580.319710,582.908822


In [28]:
big_df = pd.concat([peaks_df, teste_df],axis=1)
big_df

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips
0,31,31.015664,1623.153269,31.188541,10243.062773,552.373258,157.062347,216.573947,552.373258,28,36,2.826510,276.186629,29.602409,32.428919
1,42,43.800501,1430.111502,44.293327,21540.729150,178.084376,12.920504,42.207336,178.084376,36,56,6.829875,89.042188,40.385563,47.215438
2,125,124.694475,1631.001467,125.249005,6686.998533,449.256447,71.048679,194.556970,194.556970,119,126,1.611050,351.977962,123.888950,125.500000
3,128,128.281527,2933.284105,127.659903,10096.715895,890.961140,484.133295,239.382450,890.961140,119,134,2.403376,445.480570,127.079839,129.483215
4,144,144.469761,1383.667055,144.548235,7786.332945,520.177021,385.848439,76.998316,520.177021,140,153,2.287660,260.088510,143.325931,145.613591
5,235,235.390313,332.094692,235.142846,7906.898565,132.552658,91.285233,0.699236,132.552658,224,238,2.232697,66.276329,234.273964,236.506662
6,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,404,1.420165,108.039156,395.310059,396.730224
7,488,487.820179,533.344351,488.149398,10970.082038,99.343812,45.262762,5.755925,99.343812,482,492,4.616647,49.671906,485.511856,490.128502
8,529,529.359782,297.023953,529.165375,4995.087932,113.048598,74.117583,42.991935,113.048598,526,532,2.244825,56.524299,528.237370,530.482195
9,582,581.614266,484.212268,581.772500,5194.024894,167.577046,11.007271,92.194617,167.577046,579,585,2.589113,83.788523,580.319710,582.908822


In [29]:
vars()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  '# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:\n# a_pattern = \'Si/SI2018/SI11318.Chn\'\n# 2022-nov-16: outros espectros:\n# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"\n# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"\n\n# a_pattern = "Eso_non_existe.Chn"\n\n# 2022-Dez-21 Buscando outro espectro\n# a_pattern = \'CCI1603-I\'\n# 2022-Dez-28 Este é um espectro mais recente:\n# a_pattern = \'SI09722.Chn\'\n# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:\n# a_pattern = \'SI06322.Chn\'',
  'import numpy as np\nfrom numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada\nfrom sklearn import linear_model\n\nimport pandas as pd\

In [30]:
nucl_iear1_df

energy  unc_en  intensity    unc_i start_level_energy  \
24na  0     472.2023  0.0014   99.90000    0.001           472.2071   
      1     996.8300     0.1    0.00210   0.0002            5235.21   
      2    1368.6250   0.005   99.99400    0.002           1368.667   
      3    2754.0080   0.011   99.86700     0.01           4122.844   
      4    2871.0000       1    0.00025  0.00004            4238.38   
...              ...     ...        ...      ...                ...   
228Ac 267    89.9540            2.01000     0.19                      
      268    93.3470            3.20000      0.3                      
      269   105.5660            1.16000     0.11                      
      270   106.8940            1.54000     0.15                      
      271   108.5800            0.39000     0.04                      

          end_level_energy multipolarity mixing_ratio unc_mr conversion_coeff  \
24na  0                  0          [M3]                 NaN          4.69E-4   
      1            4238.38                               NaN                    
      2                  0            E2                 NaN           1.3E-5   
      3           1368.667            E2                 NaN                    
      4           1368.667         M1+E2          -23      9                    
...                    ...           ...          ...    ...              ...   
228Ac 267                                                                       
      268                                                                       
      269                                                                       
      270                                                                       
      271                                                                       

           ...         q  unc_q  d_z  d_n d_symbol  ensdf_publication_cut-off  \
24na  0    ...              NaN   11   13       Na                31-Mar-2022   
      1    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      2    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      3    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      4    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
...        ...       ...    ...  ...  ...      ...                        ...   
228Ac 267  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      268  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      269  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      270  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      271  ...    2123.8   26.0   90  138       Th                31-Dec-2012   

                                            ensdf_authors Extraction_date  \
24na  0    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      1    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      2    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      3    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      4    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
...                                                   ...             ...   
228Ac 267                              KHALIFEH ABUSALEEM      2023-06-15   
      268                              KHALIFEH ABUSALEEM      2023-06-15   
      269                              KHALIFEH ABUSALEEM      2023-06-15   
      270                              KHALIFEH ABUSALEEM      2023-06-15   
      271                              KHALIFEH ABUSALEEM      2023-06-15   

           nuclide_name is_to_consider  
24na  0            24na           True  
      1            24na           True  
      2            24na           True  
      3            24na           True  
      4            24na           True  
...                 ...            ...  
228

In [31]:
teste1_df = nucl_iear1_df.loc [
    (nucl_iear1_df.intensity > 1.0) & nucl_iear1_df.is_to_consider
]
teste1_df

energy  unc_en  intensity  unc_i start_level_energy  \
24na  0     472.2023  0.0014     99.900  0.001           472.2071   
      2    1368.6250   0.005     99.994  0.002           1368.667   
      3    2754.0080   0.011     99.867   0.01           4122.844   
51cr  0     320.0824  0.0004      9.910   0.01           320.0835   
54mn  0     834.8480   0.003     99.976  0.001            834.848   
...              ...     ...        ...    ...                ...   
228Ac 145   968.9710   0.017     15.800    0.3            968.972   
      215  1588.2000    0.03      3.220   0.08           1646.005   
      218  1630.6270    0.01      1.510   0.04           1688.398   
      269   105.5660              1.160   0.11                      
      270   106.8940              1.540   0.15                      

          end_level_energy multipolarity mixing_ratio unc_mr conversion_coeff  \
24na  0                  0          [M3]                 NaN          4.69E-4   
      2                  0            E2                 NaN           1.3E-5   
      3           1368.667            E2                 NaN                    
51cr  0                0.0         M1+E2         0.47      3          0.00181   
54mn  0                0.0            E2                 NaN          2.40E-4   
...                    ...           ...          ...    ...              ...   
228Ac 145              0.0            E2                 NaN          0.01061   
      215           57.763            E2                 NaN          0.00431   
      218           57.763       (M1 E2)                 NaN            0.007   
      269                                                                       
      270                                                                       

           ...         q  unc_q  d_z  d_n d_symbol  ensdf_publication_cut-off  \
24na  0    ...              NaN   11   13       Na                31-Mar-2022   
      2    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      3    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
51cr  0    ...    752.39   19.0   23   28        V                 1-Mar-2016   
54mn  0    ...    1377.1   10.0   24   30       Cr                20-Jun-2014   
...        ...       ...    ...  ...  ...      ...                        ...   
228Ac 145  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      215  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      218  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      269  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      270  ...    2123.8   26.0   90  138       Th                31-Dec-2012   

                                            ensdf_authors Extraction_date  \
24na  0    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      2    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      3    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
51cr  0                     Wang Jimin and Huang Xiaolong      2023-06-15   
54mn  0                          YANG DONG and  HUO JUNDE      2023-06-15   
...                                                   ...             ...   
228Ac 145                              KHALIFEH ABUSALEEM      2023-06-15   
      215                              KHALIFEH ABUSALEEM      2023-06-15   
      218                              KHALIFEH ABUSALEEM      2023-06-15   
      269                              KHALIFEH ABUSALEEM      2023-06-15   
      270                              KHALIFEH ABUSALEEM      2023-06-15   

           nuclide_name is_to_consider  
24na  0            24na           True  
      2            24na           True  
      3            24na           True  
51cr  0            51cr           True  
54mn  0            54mn           True  
...                 ...            ...  
228Ac 145         228Ac    

In [32]:
big_df

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips
0,31,31.015664,1623.153269,31.188541,10243.062773,552.373258,157.062347,216.573947,552.373258,28,36,2.826510,276.186629,29.602409,32.428919
1,42,43.800501,1430.111502,44.293327,21540.729150,178.084376,12.920504,42.207336,178.084376,36,56,6.829875,89.042188,40.385563,47.215438
2,125,124.694475,1631.001467,125.249005,6686.998533,449.256447,71.048679,194.556970,194.556970,119,126,1.611050,351.977962,123.888950,125.500000
3,128,128.281527,2933.284105,127.659903,10096.715895,890.961140,484.133295,239.382450,890.961140,119,134,2.403376,445.480570,127.079839,129.483215
4,144,144.469761,1383.667055,144.548235,7786.332945,520.177021,385.848439,76.998316,520.177021,140,153,2.287660,260.088510,143.325931,145.613591
5,235,235.390313,332.094692,235.142846,7906.898565,132.552658,91.285233,0.699236,132.552658,224,238,2.232697,66.276329,234.273964,236.506662
6,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,404,1.420165,108.039156,395.310059,396.730224
7,488,487.820179,533.344351,488.149398,10970.082038,99.343812,45.262762,5.755925,99.343812,482,492,4.616647,49.671906,485.511856,490.128502
8,529,529.359782,297.023953,529.165375,4995.087932,113.048598,74.117583,42.991935,113.048598,526,532,2.244825,56.524299,528.237370,530.482195
9,582,581.614266,484.212268,581.772500,5194.024894,167.577046,11.007271,92.194617,167.577046,579,585,2.589113,83.788523,580.319710,582.908822


In [33]:
def spec_engy_identif(peaks_net_kev_df, nucl_lib_df, func_en,
                      en_toler_calib=3.0, en_toler_ident=0.5, min_intensity = 1.0):
    # Library-based spectrum energy/channel recalibration
    # TODO: implement option to update self.channel_energy_calib

    """Perform a library-based spectrum energy/channel recalibration
    :param func_en: energy function
    :type callable
    :param initial_energy: column specifying the base energy values to consider
    :type initial_energy: string
    :param peaks_net_kev_df: single col 'energy_detected' w/ peaks' energies detected
    :type peaks_net_kev_df: pd.Dataframe
    :param nucl_lib_df: nuclide library based on IAEA format as a Pandas dataframe
    :type nucl_lib_df: pd.Dataframe
    :param en_toler_calib: energy difference accetable to take a peak for automatic calibration
        (default is 3.0)
    :type en_toler_calib: float
    :param en_toler_ident: energy difference accetable to take a detected peak as possibly identified
        (default is 0.5)
    :type en_toler_ident: float
    :param min_intensity: minimum yield percentage to accept a peak
        (default is 1.0)
    :type min_intensity: float
    :returns: a pd.Dataframe with each row as an identified peak with parameters
    :rtype: pd.Dataframe
    """

    left = peaks_net_kev_df
    left['engy_pk_det'] = func_en(left.centroids)
    right = nucl_lib_df.loc [
        (nucl_lib_df.intensity > min_intensity) & nucl_lib_df.is_to_consider
    ]
    cross_df = pd.merge(left, right, how='cross')
    cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
    df1 = pd.DataFrame(cross_df.loc [(np.abs(cross_df.delta_en) < en_toler_calib)] )
    X_energy = np.array(df1.energy).reshape(-1,1)
    y_delta_en = np.array(df1.delta_en)
    # Robustly fit linear model with RANSAC algorithm
    ransac = linear_model.RANSACRegressor()
    ransac.fit(X_energy, y_delta_en)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
    df1["recalib_engy_ransac"] = df1.engy_pk_det - ransac.predict(X_energy)
    # cross_df["recalib_engy_ransac"] = cross_df.engy_pk_det - ransac.predict(X_energy)

    cross_df_2 = pd.merge(df1, right, how='cross', suffixes=('_x', None))
    cross_df_2.drop(cross_df_2.filter(regex='_x$').columns, axis=1, inplace=True)
    cross_df_2["new_delta_en"] = cross_df_2.recalib_engy_ransac - cross_df_2.energy
    df_result = pd.DataFrame(cross_df_2.loc [(np.abs(cross_df_2.new_delta_en) < en_toler_ident)] )
    return left, right, df1, cross_df_2, df_result
    # return cross_df_2

In [34]:
d1, d2, d3, d4, d5 = spec_engy_identif(
    big_df, nucl_iear1_df, a_spec.channel_energy_calib.get_energy
)

In [35]:
d1

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips,engy_pk_det
0,31,31.015664,1623.153269,31.188541,10243.062773,552.373258,157.062347,216.573947,552.373258,28,36,2.826510,276.186629,29.602409,32.428919,17.066959
1,42,43.800501,1430.111502,44.293327,21540.729150,178.084376,12.920504,42.207336,178.084376,36,56,6.829875,89.042188,40.385563,47.215438,25.074512
2,125,124.694475,1631.001467,125.249005,6686.998533,449.256447,71.048679,194.556970,194.556970,119,126,1.611050,351.977962,123.888950,125.500000,74.541698
3,128,128.281527,2933.284105,127.659903,10096.715895,890.961140,484.133295,239.382450,890.961140,119,134,2.403376,445.480570,127.079839,129.483215,76.014854
4,144,144.469761,1383.667055,144.548235,7786.332945,520.177021,385.848439,76.998316,520.177021,140,153,2.287660,260.088510,143.325931,145.613591,86.334306
5,235,235.390313,332.094692,235.142846,7906.898565,132.552658,91.285233,0.699236,132.552658,224,238,2.232697,66.276329,234.273964,236.506662,141.691269
6,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,404,1.420165,108.039156,395.310059,396.730224,240.142348
7,488,487.820179,533.344351,488.149398,10970.082038,99.343812,45.262762,5.755925,99.343812,482,492,4.616647,49.671906,485.511856,490.128502,296.288482
8,529,529.359782,297.023953,529.165375,4995.087932,113.048598,74.117583,42.991935,113.048598,526,532,2.244825,56.524299,528.237370,530.482195,321.350900
9,582,581.614266,484.212268,581.772500,5194.024894,167.577046,11.007271,92.194617,167.577046,579,585,2.589113,83.788523,580.319710,582.908822,353.495976


In [36]:
d2

energy  unc_en  intensity  unc_i start_level_energy  \
24na  0     472.2023  0.0014     99.900  0.001           472.2071   
      2    1368.6250   0.005     99.994  0.002           1368.667   
      3    2754.0080   0.011     99.867   0.01           4122.844   
51cr  0     320.0824  0.0004      9.910   0.01           320.0835   
54mn  0     834.8480   0.003     99.976  0.001            834.848   
...              ...     ...        ...    ...                ...   
228Ac 145   968.9710   0.017     15.800    0.3            968.972   
      215  1588.2000    0.03      3.220   0.08           1646.005   
      218  1630.6270    0.01      1.510   0.04           1688.398   
      269   105.5660              1.160   0.11                      
      270   106.8940              1.540   0.15                      

          end_level_energy multipolarity mixing_ratio unc_mr conversion_coeff  \
24na  0                  0          [M3]                 NaN          4.69E-4   
      2                  0            E2                 NaN           1.3E-5   
      3           1368.667            E2                 NaN                    
51cr  0                0.0         M1+E2         0.47      3          0.00181   
54mn  0                0.0            E2                 NaN          2.40E-4   
...                    ...           ...          ...    ...              ...   
228Ac 145              0.0            E2                 NaN          0.01061   
      215           57.763            E2                 NaN          0.00431   
      218           57.763       (M1 E2)                 NaN            0.007   
      269                                                                       
      270                                                                       

           ...         q  unc_q  d_z  d_n d_symbol  ensdf_publication_cut-off  \
24na  0    ...              NaN   11   13       Na                31-Mar-2022   
      2    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      3    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
51cr  0    ...    752.39   19.0   23   28        V                 1-Mar-2016   
54mn  0    ...    1377.1   10.0   24   30       Cr                20-Jun-2014   
...        ...       ...    ...  ...  ...      ...                        ...   
228Ac 145  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      215  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      218  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      269  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      270  ...    2123.8   26.0   90  138       Th                31-Dec-2012   

                                            ensdf_authors Extraction_date  \
24na  0    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      2    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      3    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
51cr  0                     Wang Jimin and Huang Xiaolong      2023-06-15   
54mn  0                          YANG DONG and  HUO JUNDE      2023-06-15   
...                                                   ...             ...   
228Ac 145                              KHALIFEH ABUSALEEM      2023-06-15   
      215                              KHALIFEH ABUSALEEM      2023-06-15   
      218                              KHALIFEH ABUSALEEM      2023-06-15   
      269                              KHALIFEH ABUSALEEM      2023-06-15   
      270                              KHALIFEH ABUSALEEM      2023-06-15   

           nuclide_name is_to_consider  
24na  0            24na           True  
      2            24na           True  
      3            24na           True  
51cr  0            51cr           True  
54mn  0            54mn           True  
...                 ...            ...  
228Ac 145         228Ac    

In [37]:
d3

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date,nuclide_name,is_to_consider,delta_en,recalib_engy_ransac
849,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,83,129,Bi,1-Aug-2020,K. Auranen and E.A. McCutchan,2023-06-15,212Pb,True,1.510348,238.908112
854,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,83,131,Bi,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Pb,True,-1.852652,238.905883
983,488,487.820179,533.344351,488.149398,10970.082038,99.343812,45.262762,5.755925,99.343812,482,...,83,131,Bi,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Pb,True,1.064482,295.016739
1027,529,529.359782,297.023953,529.165375,4995.087932,113.048598,74.117583,42.991935,113.048598,526,...,23,28,V,1-Mar-2016,Wang Jimin and Huang Xiaolong,2023-06-15,51cr,True,1.268500,320.062682
1240,582,581.614266,484.212268,581.772500,5194.024894,167.577046,11.007271,92.194617,167.577046,579,...,83,131,Bi,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Pb,True,1.563976,352.186650
1326,715,715.115733,139.405831,715.042172,2414.594169,71.516254,49.552176,37.518001,71.516254,712,...,57,83,La,20-Nov-2018,N. NICA,2023-06-15,140ba,True,-2.645876,433.563038
1329,715,715.115733,139.405831,715.042172,2414.594169,71.516254,49.552176,37.518001,71.516254,712,...,58,82,Ce,20-Nov-2018,N. NICA,2023-06-15,140la,True,2.436124,433.566406
1613,960,960.262753,405.946228,960.212406,2882.701328,151.182895,73.438463,26.600099,151.182895,951,...,82,126,Pb,1-Jun-2007,M. J. MARTIN,2023-06-15,208Tl,True,1.551030,583.275440
1676,1003,1002.991297,698.096867,1002.890168,2736.208109,276.941300,121.510938,121.636425,276.941300,983,...,45,58,Rh,1-Mar-2009,D. DE FRENNE,2023-06-15,103ru,True,0.482865,609.335284
1754,1003,1002.991297,698.096867,1002.890168,2736.208109,276.941300,121.510938,121.636425,276.941300,983,...,84,130,Po,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Bi,True,1.494865,609.335954


In [38]:
d4

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date,nuclide_name,is_to_consider,new_delta_en
0,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,NaN,11,13,Na,31-Mar-2022,M. Shamsuzzoha Basunia and Anagha Chakraborty,2023-06-15,24na,True,-233.294188
1,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,21.0,12,12,Mg,31-Mar-2022,M. Shamsuzzoha Basunia and Anagha Chakraborty,2023-06-15,24na,True,-1129.716888
2,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,21.0,12,12,Mg,31-Mar-2022,M. Shamsuzzoha Basunia and Anagha Chakraborty,2023-06-15,24na,True,-2515.099888
3,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,19.0,23,28,V,1-Mar-2016,Wang Jimin and Huang Xiaolong,2023-06-15,51cr,True,-81.174288
4,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,10.0,24,30,Cr,20-Jun-2014,YANG DONG and HUO JUNDE,2023-06-15,54mn,True,-595.939888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,4014,4012.617584,175.720340,4011.784422,384.509923,18.789239,4.999285,3.001582,18.789239,3998,...,26.0,90,138,Th,31-Dec-2012,KHALIFEH ABUSALEEM,2023-06-15,228Ac,True,1477.702385
5756,4014,4012.617584,175.720340,4011.784422,384.509923,18.789239,4.999285,3.001582,18.789239,3998,...,26.0,90,138,Th,31-Dec-2012,KHALIFEH ABUSALEEM,2023-06-15,228Ac,True,858.473385
5757,4014,4012.617584,175.720340,4011.784422,384.509923,18.789239,4.999285,3.001582,18.789239,3998,...,26.0,90,138,Th,31-Dec-2012,KHALIFEH ABUSALEEM,2023-06-15,228Ac,True,816.046385
5758,4014,4012.617584,175.720340,4011.784422,384.509923,18.789239,4.999285,3.001582,18.789239,3998,...,26.0,90,138,Th,31-Dec-2012,KHALIFEH ABUSALEEM,2023-06-15,228Ac,True,2341.107385


In [39]:
d5

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date,nuclide_name,is_to_consider,new_delta_en
81,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,18.0,83,129,Bi,1-Aug-2020,K. Auranen and E.A. McCutchan,2023-06-15,212Pb,True,0.276112
209,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,18.0,83,129,Bi,1-Aug-2020,K. Auranen and E.A. McCutchan,2023-06-15,212Pb,True,0.273883
343,488,487.820179,533.344351,488.149398,10970.082038,99.343812,45.262762,5.755925,99.343812,482,...,11.0,83,131,Bi,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Pb,True,-0.207261
387,529,529.359782,297.023953,529.165375,4995.087932,113.048598,74.117583,42.991935,113.048598,526,...,19.0,23,28,V,1-Mar-2016,Wang Jimin and Huang Xiaolong,2023-06-15,51cr,True,-0.019718
600,582,581.614266,484.212268,581.772500,5194.024894,167.577046,11.007271,92.194617,167.577046,579,...,11.0,83,131,Bi,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Pb,True,0.254650
973,960,960.262753,405.946228,960.212406,2882.701328,151.182895,73.438463,26.600099,151.182895,951,...,17.0,82,126,Pb,1-Jun-2007,M. J. MARTIN,2023-06-15,208Tl,True,0.088440
1114,1003,1002.991297,698.096867,1002.890168,2736.208109,276.941300,121.510938,121.636425,276.941300,983,...,11.0,84,130,Po,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Bi,True,0.014284
1242,1003,1002.991297,698.096867,1002.890168,2736.208109,276.941300,121.510938,121.636425,276.941300,983,...,11.0,84,130,Po,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Bi,True,0.014954
1295,1082,1082.174080,1959.424012,1082.105291,4312.575988,662.363124,273.496198,118.578263,662.363124,1077,...,13.0,48,62,Cd,1-Aug-2011,G. G~URDAL and F.G. KONDEV,2023-06-15,110ag,True,0.207660
1296,1082,1082.174080,1959.424012,1082.105291,4312.575988,662.363124,273.496198,118.578263,662.363124,1077,...,13.0,48,62,Cd,1-Aug-2011,G. G~URDAL and F.G. KONDEV,2023-06-15,110ag,True,-0.052340


In [40]:
d5.columns

Index(['peaks', 'fwhm_centr', 'rough_sums', 'centroids', 'variances', 'pk_hei',
       'lf_thr', 'rg_thr', 'promns', 'lf_bas', 'rg_bas', 'widths', 'wi_hei',
       'lf_ips', 'rg_ips', 'engy_pk_det', 'delta_en', 'recalib_engy_ransac',
       'energy', 'unc_en', 'intensity', 'unc_i', 'start_level_energy',
       'end_level_energy', 'multipolarity', 'mixing_ratio', 'unc_mr',
       'conversion_coeff', 'unc_cc', 'p_z', 'p_n', 'p_symbol',
       'p_energy_shift', 'p_energy', 'unc_pe', 'jp', 'half_life',
       'operator_hl', 'unc_hl', 'unit_hl', 'half_life_sec', 'unc_hls', 'decay',
       'decay_%', 'unc_d', 'q', 'unc_q', 'd_z', 'd_n', 'd_symbol',
       'ensdf_publication_cut-off', 'ensdf_authors', 'Extraction_date',
       'nuclide_name', 'is_to_consider', 'new_delta_en'],
      dtype='object')

In [41]:
# 2023-Jun-20: Cálculo das atividades

In [43]:
# Ver exatamente o que são esses elementos de Polynomial

In [42]:
# 2023-Jun-14
# Loading calibration pkl file:
calib_pkl_name = 'f100_gmx_2021.pkl'
calib_df = pd.read_pickle(calib_pkl_name)
calib_df

,Engy,Effic,sEffic,effic_func
0,59.54,6.4299,3.61,743
1,81.00,5.6528,3.61,743
2,276.40,1.6306,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
3,302.85,1.5006,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
4,356.02,1.4023,3.61,743
5,383.85,1.3216,3.61,743
6,604.69,0.9112,3.61,743
7,661.66,0.7900,3.61,743
8,795.84,0.7000,3.61,743
9,1155.00,0.5916,0.01,743


In [44]:
p_eff, _ = calib_df.effic_func[3]
p_eff

Polynomial([ 0.51104917, -1.54282532,  0.42082601,  0.36310567, -0.3132035 ,
       -0.06636575], domain=[4.08664836, 7.19481216], window=[-1.,  1.])

In [50]:
def en_eff(engy):
    return np.exp(p_eff(np.log(engy)))

In [52]:
en_eff(1332)

0.5341252964668152

In [61]:
d5["disintegr"] = d5.rough_sums / (en_eff(d5.energy) * 1e-4 * d5.intensity)
d6 = d5[key='nuclide_name',['energy','intensity','centroids','rough_sums','variances','disintegr']]
d6

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2182989551.py, line 2)

In [ ]:
# To get activities/concentration, one uses:

In [53]:
a_spec.lv_time

50000.0

In [ ]:
# Sample size (L, kg, g etc.)
samp_size = 0.1

In [54]:
a_spec.sam_descr

'SI-063/22 - Agua da Piscina do Reator IEA-R1 (GMX)'

In [55]:
a_spec.source_datetime

In [56]:
a_spec.start_datetime

datetime.datetime(2022, 8, 4, 19, 42, 23)

In [ ]:
# 2023-Mar-22 PAREI AQUI

In [ ]:
f hjrdft jrdjhrstjh rsthj s

In [ ]:
# 2023-May-24
# Implementing:
#  - nuclide labels
# https://plotly.com/python/text-and-annotations/
#  - and energy search in pandas
# https://stackoverflow.com/questions/63529555/get-list-of-occurrences-using-pandas

In [ ]:
# counts_nzero_cps = counts_nzero / a_spec.lv_time
# unc_y_cps = unc_y / a_spec.lv_time
# fin_bl_cps = fin_bl / a_spec.lv_time
# counts_cps = counts / a_spec.lv_time
# en_kev_nzero = get_energy(chans_nzero)
# en_kev = get_energy(chans)
# peaks_net_kev = get_energy(peaks_net)

In [58]:
help(spec_engy_identif)

Help on function spec_engy_identif in module __main__:

spec_engy_identif(peaks_net_kev_df, nucl_lib_df, func_en, en_toler_calib=3.0, en_toler_ident=0.5, min_intensity=1.0)
    Perform a library-based spectrum energy/channel recalibration
    :param func_en: energy function
    :type callable
    :param initial_energy: column specifying the base energy values to consider
    :type initial_energy: string
    :param peaks_net_kev_df: single col 'energy_detected' w/ peaks' energies detected
    :type peaks_net_kev_df: pd.Dataframe
    :param nucl_lib_df: nuclide library based on IAEA format as a Pandas dataframe
    :type nucl_lib_df: pd.Dataframe
    :param en_toler_calib: energy difference accetable to take a peak for automatic calibration
        (default is 3.0)
    :type en_toler_calib: float
    :param en_toler_ident: energy difference accetable to take a detected peak as possibly identified
        (default is 0.5)
    :type en_toler_ident: float
    :param min_intensity: minimum y

In [ ]:
# big_df = identified_peaks_df

aç~]sldgfk~çlaskg~çl

# AQUI: fazer o spec_engy_identif completo

In [ ]:
# 2023-Jun-13
# PAREI AQUI
# Trazer o trabalho feito em
# radionuclide_analysis.ipynb e adaptar ao que está aqui
# especialmente na identificaćão dos nucl[ideos

# Depois, deletar radionuclide_analysis.ipynb.

In [ ]:
xdfgh rxfth rxthj rds

In [62]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)


df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)


frames = [df1, df2, df3]

result = pd.concat(frames)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


In [63]:
result = pd.concat(frames, keys=["x", "y", "z"])
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [64]:
# https://stackoverflow.com/questions/63529555

In [65]:
animals = ['cat', 'dog', 'hamster', 'dolphin']

In [66]:
df_animals_sets = pd.DataFrame(data={
    'id': [1,2,3,4,5],
    'animals': ['dog,cat','dog','cat,dolphin','cat,dog','hamster,dolphin']
})
df_animals_sets

,id,animals
0,1,"dog,cat"
1,2,dog
2,3,"cat,dolphin"
3,4,"cat,dog"
4,5,"hamster,dolphin"


In [67]:
df = df_animals_sets
df_final = (df.astype(str).assign(animals=df.animals.str.split(','))
                          .explode('animals').groupby('animals').id.agg(','.join)
                          .reset_index())
df_final

,animals,id
0,cat,"1,3,4"
1,dog,"1,2,4"
2,dolphin,"3,5"
3,hamster,5
